### Importing the Neccessary Modules

In [5]:
# Importing the Necessary Modules for Wallet Explorer for Realized and Unrealized PnL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import json
import datetime
import time 
from __future__ import annotations
from dataclasses import dataclass
from enum import Enum
from typing import List, Dict, Optional, Tuple
from dotenv import load_dotenv
from dune_client.client import DuneClient



In [6]:
# Load query output from Dune

load_dotenv()
API_KEY = os.getenv('DUNE_CLIENT_API')

from dune_client.client import DuneClient
dune = DuneClient(API_KEY)
response = dune.get_custom_endpoint_result(
  "firstbml",
  "wallet-explorer-drilldown",
  limit = 100
)

2025-08-28 23:33:32,939 ERROR dune_client.models Can't build ResultsResponse from {'error': 'invalid API Key'} due to KeyError: 'execution_id'


DuneError: Can't build ResultsResponse from {'error': 'invalid API Key'}

In [3]:
from dune_client.client import DuneClient
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("DUNE_CLIENT_API")

print("Using API Key:", api_key[:6] + "..." if api_key else "None found")

dune = DuneClient(api_key)


Using API Key: None found


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
print(repr(os.getenv("DUNE_CLIENT_API")))


None
